🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# LangGraph Multi-Agent Yelp Navigator

An educational example of building a multi-agent system using LangGraph to create an intelligent Yelp business search assistant.

## What You'll Learn

- How to orchestrate multiple specialized nodes working together
- How to implement conditional routing based on user needs
- How to integrate external tools and APIs into LangGraph workflows
- How to add quality control through a supervisor pattern

## Architecture

The system has **6 specialized nodes**:

1. **Clarification Node** - Extracts query, location, and detail level from user input
2. **Search Node** - Finds businesses matching the criteria - uses the Haystack `business_search` endpoint as a tool
3. **Details Node** - Fetches additional info (websites, descriptions) - uses the Haystack `business_details` endpoint as a tool
4. **Sentiment Node** - Analyzes customer reviews - uses the Haystack  `business_sentiment` endpoint as a tool
5. **Summary Node** - Creates a user-friendly response
6. **Supervisor Approval Node** - Reviews quality and can request revisions


- [Tool implementation](./langgraph_helpers/tools.py)
- [Node implementation](./langgraph_helpers/nodes.py)

The system allows to flexibly choose the best endpoint for the query and routes back if there is incomplete information (determined by the approval node)


## Prerequisites

**Start Hayhooks server:**
```bash
cd yelp-navigator
uv run sh build_all_pipelines.sh && sh start_hayhooks.sh
```

### Optional 
To run the agent locally, you can use Ollama and GPT-OSS 

```
ollama pull gpt-oss:20b-latest
```

A code snippet is provided and commented out where you can swap the LLM from OpenAI to Ollama with GPT-OSS.

In [1]:
# =============================================================================
# STEP 1: Imports and Configuration
# =============================================================================

import os
import requests
from typing import Dict, Any

# LangGraph for building agent workflows
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
from pathlib import Path

# Optional if you don't want to use OpenAI
from langchain_ollama import ChatOllama

# Load .env from the root of ch8 directory
root_dir = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
load_dotenv(root_dir / ".env")

# Import custom agent logic from helper modules
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'langgraph-helpers'))
from langgraph_helpers.nodes import clarification_node, supervisor_approval_node, summary_node
from langgraph_helpers.nodes import search_node, details_node, sentiment_node
from langgraph_helpers.tools import search_businesses, get_business_details, analyze_reviews_sentiment, set_base_url

# Configure endpoints
BASE_URL = "http://localhost:1416"  # Hayhooks server
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

set_base_url(BASE_URL)

if not OPENAI_API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not set. Please add it to your .env file.")
else:
    print("✅ OpenAI API key configured")
    print(f"✅ Hayhooks server: {BASE_URL}")

✅ OpenAI API key configured
✅ Hayhooks server: http://localhost:1416


## Test Server Connection



In [2]:
# Check if Hayhooks server is running
try:
    response = requests.get(f"{BASE_URL}/status")
    print("Hayhooks server is running!")
    print(f"Status: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("Cannot connect to Hayhooks server")
    print("Please start the server with: hayhooks run --pipelines-dir pipelines")

Hayhooks server is running!
Status: 200


**If you do not get 200, please ensure you're running Hayhooks on your terminal**



## Multi-Agent Supervisor Pattern

Using LangGraph to build stateful, cyclic workflows with LLMs.

### Key Concepts

- **MessagesState**: Automatic message history handling with `add_messages` reducer
- **Conditional Routing**: Dynamic workflows based on state values
- **Supervisor Pattern**: Quality control that reviews outputs and requests revisions

In [3]:
# =============================================================================
# STEP 2: Define Agent State
# =============================================================================
# AgentState inherits from MessagesState to get automatic message handling.
# MessagesState provides:
# - 'messages' field with add_messages reducer (appends new messages, updates by ID)
# - Support for message shorthand formats (e.g., OpenAI format)
# - Automatic message ID tracking for updates
#
# Each node reads from and writes to this state as the workflow progresses.

class AgentState(MessagesState):
    """State shared across all nodes in the workflow.
    
    Inherits 'messages: Annotated[List[BaseMessage], add_messages]' from MessagesState.
    """
    
    # User intent (set by Clarification Node)
    user_query: str              # Original user question
    clarified_query: str         # What they're looking for (e.g., "Mexican restaurants")
    clarified_location: str      # Where to search (e.g., "Austin, Texas")
    detail_level: str            # "general", "detailed", or "reviews"
    
    # Workflow control
    clarification_complete: bool  # True when ready to proceed to search
    next_agent: str              # Which node to call next (enables conditional routing)
    
    # Node results
    agent_outputs: Dict[str, Any]  # Results from each node (search, details, sentiment)
    
    # Final output
    final_summary: str  # User-friendly response
    
    # Quality control (for Supervisor Approval Node)
    approval_attempts: int      # How many times supervisor has reviewed (max 2)
    needs_revision: bool        # True if summary needs improvement
    revision_feedback: str      # What to improve

## State Management

`MessagesState` provides automatic message handling with `add_messages` reducer:
- Appends new messages
- Updates existing messages by ID
- Prevents accidental overwriting

**Custom Fields:** `user_query`, `clarified_query`, `clarified_location`, `detail_level`, `agent_outputs`, `final_summary`

## Initialize Language Model

In [4]:
# Initialize the LLM that will power our agents
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# NOTE: A LOCAL MODEL WILL BE MUCH SLOWER THAN OPENAI'S API
# llm = ChatOllama(
#                 model="gpt-oss:20b",
#                 temperature=0,
#                 timeout=60,
#             )

## Node Wrappers

Inject dependencies into node logic:

**LLM-Powered:** Clarification, Summary, Supervisor  
**Tool-Powered:** Search, Details, Sentiment (call Hayhooks pipelines)

### LLM-Powered Nodes

In [5]:
# =============================================================================
# STEP 4: Create Node Wrapper Functions
# =============================================================================
# Wrappers pass the LLM to nodes that need it for reasoning

def clarification_wrapper(state: AgentState) -> AgentState:
    """Extracts query, location, and detail level from user input."""
    return clarification_node(state, llm)

def supervisor_approval_wrapper(state: AgentState) -> AgentState:
    """Reviews summary quality and decides if revision is needed."""
    return supervisor_approval_node(state, llm)

def summary_wrapper(state: AgentState) -> AgentState:
    """Synthesizes all information into a user-friendly response."""
    return summary_node(state, llm)

### Tool-powered nodes

In [6]:
# Wrappers pass tools to nodes that interact with external pipelines

def search_wrapper(state: AgentState) -> AgentState:
    """Searches for businesses using the Hayhooks search pipeline."""
    return search_node(state, search_businesses)

def details_wrapper(state: AgentState) -> AgentState:
    """Fetches business details (website, description) via Hayhooks."""
    return details_node(state, get_business_details)

def sentiment_wrapper(state: AgentState) -> AgentState:
    """Analyzes review sentiment via Hayhooks sentiment pipeline."""
    return sentiment_node(state, analyze_reviews_sentiment)

## Routing Logic

Routing functions decide which node runs next:

- **After Clarification:** Loop until all info gathered
- **After Search:** Skip details/sentiment if `detail_level == "general"`
- **After Details:** Route to sentiment only if `detail_level == "reviews"`
- **After Supervisor:** Approve (END) or revise (route to any node)

```python
workflow.add_conditional_edges("search", route_fn, destinations)
```

In [7]:
# =============================================================================
# STEP 5: Define Routing Logic and Build the Workflow
# =============================================================================

def route_after_clarification(state: AgentState) -> str:
    """Route to search if clarification is complete, otherwise continue clarifying."""
    if state.get("clarification_complete", False):
        return "search"
    return "clarification"


def route_after_search(state: AgentState) -> str:
    """Route based on detail level: general->summary, detailed/reviews->details."""
    detail_level = state.get("detail_level", "general")
    
    if detail_level == "general":
        return "summary"
    else:  # detailed or reviews - both need details first
        return "details"


def route_after_details(state: AgentState) -> str:
    """Route from details node based on detail level."""
    detail_level = state.get("detail_level", "general")
    
    if detail_level == "reviews":
        return "sentiment"
    else:
        return "summary"


def route_from_supervisor_approval(state: AgentState) -> str:
    """Route from supervisor approval to either END or back to a node."""
    next_agent = state.get("next_agent", "end")
    
    if next_agent == "end":
        return END
    
    # Supervisor wants to rerun a node
    return next_agent


## Build graph

In [8]:
def build_workflow_graph() -> StateGraph[AgentState]:
    """Builds a true pipeline architecture for Yelp Navigator.
    
    Pipeline Flow:
    1. Clarify → Search (sequential, clarify may loop)
    2. Search → Sentiment OR Details OR Summary (based on detail_level)
    3. Details → Sentiment OR Summary (based on detail_level)
    4. Sentiment → Summary (always)
    5. Summary → Approval (always)
    6. Approval → END or back to any step for revision
    """
    # Build the graph
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("clarification", clarification_wrapper)
    workflow.add_node("search", search_wrapper)
    workflow.add_node("details", details_wrapper)
    workflow.add_node("sentiment", sentiment_wrapper)
    workflow.add_node("summary", summary_wrapper)
    workflow.add_node("supervisor_approval", supervisor_approval_wrapper)
    
    # Add edges - TRUE PIPELINE ARCHITECTURE
    # START -> Clarification (may loop back to itself)
    workflow.add_edge(START, "clarification")

    # Clarification loops or moves to search
    workflow.add_conditional_edges(
        "clarification",
        route_after_clarification,
        {"clarification": "clarification", "search": "search"}
    )

    # Search -> Details OR Summary (based on detail level)
    workflow.add_conditional_edges(
        "search",
        route_after_search,
        {"details": "details", "summary": "summary"}
    )

    # Details -> Sentiment OR Summary (based on detail level)
    workflow.add_conditional_edges(
        "details",
        route_after_details,
        {"sentiment": "sentiment", "summary": "summary"}
    )

    # Sentiment -> Summary (always)
    workflow.add_edge("sentiment", "summary")

    # Summary -> Supervisor Approval (always)
    workflow.add_edge("summary", "supervisor_approval")

    # Supervisor Approval can route back to nodes or to END
    workflow.add_conditional_edges(
        "supervisor_approval",
        route_from_supervisor_approval,
        {
            "search": "search",
            "details": "details", 
            "sentiment": "sentiment",
            "summary": "summary",
            END: END
        }
    )
    return workflow

# Compile the graph with increased recursion limit for approval loops
graph = build_workflow_graph().compile()

## Graph Visualization

Shows nodes, edges, conditional routing points, and feedback loops (clarification loop, supervisor revisions).

In [11]:
# Visualize the workflow graph
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except ImportError:
    # Fallback: print text representation
    print("Graph visualization requires additional dependencies.")
    print("\nGraph structure:")
    print(graph.get_graph())
except Exception as e:
    print(f"Could not display graph: {e}")
    print("\nAlternatively, you can view the saved image at ./images/langgraph-supervisor.png")

Could not display graph: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

Alternatively, you can view the saved image at ./images/langgraph-supervisor.png


![](./images/langgraph-supervisor.png)

## Execution

`graph.stream()` executes the workflow step-by-step. Different paths based on `detail_level`:
- `"general"` → skip details/sentiment
- `"detailed"` → include details
- `"reviews"` → full pipeline

In [10]:
# Example 1: General search - test with your own query
query = "Find me pizza places in Chicago"

print(f"🔍 Searching: {query}\n")

# Run the workflow
for step in graph.stream({"user_query": query}):
    for node_name, output in step.items():
        if output.get("messages"):
            msg = output["messages"][-1].content
            # Show full summary, truncate other nodes
            if node_name == "summary" or node_name == "supervisor_approval":
                print(f"[{node_name}]\n{msg}\n")
            else:
                print(f"[{node_name}] {msg[:200]}{'...' if len(msg) > 200 else ''}\n")

print("✅ Done!")


🔍 Searching: Find me pizza places in Chicago

[clarification] CLARIFIED:
QUERY: pizza places
LOCATION: Chicago
DETAIL_LEVEL: general

[search_businesses] Calling API with query: pizza places in Chicago
[search] Found 0 businesses:


[summary]
SUMMARY:

Chicago is famous for its diverse and delicious pizza scene, offering everything from deep-dish to thin-crust and everything in between. While I don't have specific business names to share, I can certainly guide you on what to expect and look for when exploring pizza in the Windy City.

First, you can't talk about Chicago pizza without mentioning the iconic deep-dish. This hearty, thick-crusted pie is loaded with cheese and chunky tomato sauce, and it's a must-try for any pizza lover. Many local spots have perfected this style, so you're sure to find a favorite.

If you're in the mood for something lighter, Chicago also offers fantastic thin-crust options. These pizzas are known for their crispy, cracker-like crusts and are often cut int

In [18]:
query = "Find italian restaurants in New York and give me their websites"

print(f"🔍 Searching: {query}\n")

# Run the workflow
for step in graph.stream({"user_query": query}):
    for node_name, output in step.items():
        if output.get("messages"):
            msg = output["messages"][-1].content
            # Show full summary, truncate other nodes
            if node_name == "summary" or node_name == "supervisor_approval":
                print(f"[{node_name}]\n{msg}\n")
            else:
                print(f"[{node_name}] {msg[:200]}{'...' if len(msg) > 200 else ''}\n")

print("✅ Done!")

🔍 Searching: Find italian restaurants in New York and give me their websites

[clarification] CLARIFIED:
QUERY: Italian restaurants
LOCATION: New York
DETAIL_LEVEL: detailed

[search_businesses] Calling API with query: Italian restaurants in New York
[search] Found 10 businesses:
1. Rubirosa - 4.3 stars (3676 reviews)
2. Da Andrea - Greenwich Village - 4.4 stars (2650 reviews)
3. Carbone - 4.0 stars (2050 reviews)
4. Olio e Più - 4.5 stars (7063 reviews)
5...

[details] Retrieved details for 0 businesses:


[summary]
SUMMARY:

If you're on the hunt for some delicious Italian cuisine in New York, you're in luck! The city is brimming with fantastic Italian restaurants that cater to all tastes and budgets. Let's dive into some top recommendations that are sure to satisfy your cravings.

First up, we have Rubirosa, a cozy spot known for its mouthwatering pizzas and classic Italian dishes. With a solid 4.3-star rating from over 3,600 reviews, it's a favorite among locals and visitors alike.

## Key Features

- Extracts query, location, detail level
- Adaptive workflow based on user needs
- Calls Haystack pipelines via Hayhooks
- Supervisor reviews with max 2 attempts
- Supports `general`, `detailed`, or `reviews` detail levels

## Design Patterns

**State Inheritance:** `class AgentState(MessagesState)` - inherit message handling  
**Dependency Injection:** Wrappers inject LLMs/tools  
**Conditional Orchestration:** Routing functions + conditional edges  
**Supervisor Approval:** Quality control with feedback loops

**Resources:** See `langgraph_helpers/` for node implementations